In [ ]:
# BCT Winter 2018
# November 7, 2018
# STATE-SPACE MODELS AND KALMAN FILTER

# APPLICATION: SIMPLE ESTIMATION OF GDP GAP
#     ... USING TREND - CYCLE MODEL

In [ ]:
#### HOUSEKEEPING ####

In [ ]:
rm(list = ls())
options(repr.plot.width = 10, repr.plot.height = 4)

In [ ]:
#### DATA ####
library(readxl)
gdp   <- na.omit(read_excel("gdpCZ.xlsx"))
dates <- gdp$period
gdp   <- log(gdp$value)
gdpG  <- c(0,diff(gdp))

In [ ]:
# Prepare nice ggplot #
library(ggplot2)
data.plot = data.frame(Date = rep(dates,2),
                       Category = rep(c("logGDP","GPDchange"), each=length(gdp)),
                       Values = c(gdp,gdpG*100))
ggplot(data=data.plot,aes(x=Date, y=Values, group=Category)) +
  geom_line(aes(color=Category)) + 
  facet_wrap(~ Category, scales='free') + 
  theme(legend.position="none")

In [ ]:
#2) let's guess the initial values of missing parametes
a1    = mean(diff(gdp)) 
a2    = 0

sigmaQ1 = log(sd(diff(gdp))^2) # will be transformed below
sigmaQ2 = log(sd(diff(gdp))^2) # will be transformed below

f22in   = 2 # will be transformed below
f23in   = 2 # will be transformed below

In [ ]:
#3) save these parameters into single vector:
theta = c(a1, a2, sigmaQ1, sigmaQ2, f22in, f23in)

In [ ]:
# 4) call data
data = gdp
pers  <- length(data)

In [ ]:
# 5) now we need to get the model matrices
HH = matrix(c(1, 1, 0), ncol = 3)
AA = matrix(c(a1, a2, 0), ncol = 1)
RR = 0

# QQ transformed to have positive diagonal elements
QQ = diag(c(exp(c(sigmaQ1, sigmaQ2)),0))

# FF matrix calculated from f22in and f23in by their
# transformation ensuring that AR(2) process is stationary
# ((note: three AR(2) stationarity conditions ensured by the
#   transformation are: f23+f22<1 , f23-f22<1 , |f23|<1 ))
f22ad = f22in/(1+abs(f22in))
f23ad = f23in/(1+abs(f23in))
f22   = f22ad + f23ad
f23   = -f22ad*f23ad
FF    = matrix(c( 1 , 0 , 0 ,
                  0 ,f22,f23,
                  0 , 1 , 0),byrow = T, ncol = 3)

In [ ]:
# 6) initial conditions for states
X0 = c(data[1]-a1,0,0) # initial trend is equal to observed GDP and initial cycle is zero
P0 = 10*QQ      # initial variance of states set rather high

In [ ]:
# 7a) prepare empty matrices for states:
statesF0 = matrix(NA,nrow = pers, ncol = 3)
statesF1 = matrix(NA,nrow = pers, ncol = 3)

# 7b) prepare empty array for state variances:
stVarsF0 = array(NA,dim = c(pers,3,3))
stVarsF1 = array(NA,dim = c(pers,3,3))

In [ ]:
# 8) go step-by-step through all periods and filter states:
for (tt in 1:pers){
  
  # predict GDP, based on the last known states
  X1pred = AA + FF%*%X0           # state prediction
  P1pred = FF%*%P0%*%t(FF) + QQ   # variance prediction
  YYpred = HH%*%X1pred            # GDP prediction
  
  statesF0[tt,]  = X1pred
  stVarsF0[tt,,] = P1pred
  
  # calculate error of prediction
  YYerror = data[tt] - YYpred     # error
  
  # updating
  obsVAR = HH%*%P1pred%*%t(HH) + RR       # predicted variance of observations
  KG     = P1pred%*%t(HH)%*%solve(obsVAR) # Kalman Gain used to update states by error
  X1     = X1pred + KG%*%YYerror          # predicted stated updated by a part of the error
  P1     = P1pred - KG%*%HH%*%P1pred      # predicted state variance updated by the KG
  # high state variance P1pred => high Kalman Gain => observed error matters (sizeable updating)
  # high variance of observations obsVAR => low Kalman Gain => measured values not so important
  # => do I believe more my predictions of states of the measurements? what has higher varinace?
  
  # saving results
  statesF1[tt,]  = X1
  stVarsF1[tt,,] = P1
  
  # going into the next loop
  X0 = X1
  P0 = P1
}

In [ ]:
#### PLOTTING RESULTS ####

In [ ]:
# CODE FOR KALMAN SMOOTHER
  # note that since we use rather simple model, the convergence
  # may imply P matrix close to singular, which would result into
  # a collapse of the function solve() to get its inverse.
  # Therefore, you may need to use function ginv() from library MASS
  # to obtain the inverse, instead.

In [ ]:
library("MASS")

statesS            = matrix(NA,nrow = pers, ncol = 3)
statesS[pers,]  = statesF1[pers,]
stVarsS            = array(NA,dim = c(pers,3,3))
stVarsS[pers,,] = stVarsF1[pers,,]

In [ ]:
# HERE COMES THE SMOOTHER RECURSION
for (tt in seq(from = pers-1, to = 1, by = -1)){
  #
  #
  #
}

In [ ]:
data.plot = data.frame(Date     = rep(dates,3),
                       Category = rep(c("Real GDP","Potential GDP","GDP Gap"), each=pers),
                       Plottt   = c(rep("GDP and Potential GDP (quarterly, mld. CZK)",2*pers),
                                  rep("GDP Gap (%)",pers)),
                       ValuesF  = c(exp(gdp),exp(statesF1[,1]),statesF1[,2]*100),
                       ValuesS  = c(exp(gdp),exp(statesS[,1]),statesS[,2]*100))
ggplot(data=data.plot,aes(x=Date, y=ValuesF, group=Category)) +
  geom_line(aes(color=Category)) + 
  facet_wrap(~ Plottt, ncol = 1, scales='free') + theme_classic() + 
  geom_hline(aes(yintercept=int), data.frame(int=c(0),Plottt=c("GDP Gap (%)")), color="red",lty=2)
ggplot(data=data.plot,aes(x=Date, y=ValuesS, group=Category)) +
  geom_line(aes(color=Category)) + 
  facet_wrap(~ Plottt, ncol = 1, scales='free') + theme_classic() + 
  geom_hline(aes(yintercept=int), data.frame(int=c(0),Plottt=c("GDP Gap (%)")), color="red",lty=2)

